In [1]:
#quickstart
from pathlib import Path
import pandas as pd
from bids import BIDSLayout
from joblib import Parallel, delayed
import nilearn
import nibabel as nib
import nltools.prefs as prf
from nilearn import datasets, image, plotting, input_data
from nilearn.masking import apply_mask
from nilearn.image import resample_to_img
from nilearn.image import concat_imgs, index_img
import numpy as np
import matplotlib.pylab as plt

import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')


import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from nltools.data import Brain_Data, Design_Matrix, Adjacency
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore
from scipy.stats import binom, ttest_1samp
from sklearn.metrics import pairwise_distances
from nltools.stats import fdr, one_sample_permutation
from copy import deepcopy
from nltools.utils import get_anatomical
import nltools.plotting as nlplt
from datetime import date
from joblib import Parallel, delayed

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

from nilearn.input_data import NiftiMapsMasker,NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

import sys
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
from pymer4 import Lmer
psych = importr('psych')
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

import os
import glob
import seaborn as sns
from statistics import mean
import matplotlib.style as style

#dat_paths = pd.read_csv('/data/MBDU/midla/notebooks/campcc/dat_paths_11272019.csv') 
#atlas = image.load_img('/data/MBDU/midla/notebooks/campcc/hox_msdl_atlas.nii.gz')
#sub1 = image.load_img('/data/MBDU/midla/data/derivatives/fmriprep/rn_aroma_20190910_2mm/sub-20900/out/fmrip

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

/gpfs/gsfs11/users/MBDU/midla/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
dat_paths = pd.read_csv('/data/MBDU/midla/data/derivatives/midla/primary_20200325/for_chris.csv')

In [3]:
# basc = datasets.fetch_atlas_basc_multiscale_2015(version='asym')
# basc64 = basc['scale064']
# atlas = image.load_img(basc64)
# atlas_dat = atlas.get_fdata()
# uvs = np.unique(atlas_dat)
# atlas_in_4d = []
# for uv in uvs:
#     atlas_in_4d.append((atlas_dat == uv).astype(int))
# atlas_4d_dat = np.array(atlas_in_4d)
# atlas_dfs = []
# for ii, aa in enumerate(atlas_in_4d):
#     row = {}
#     row['orig_ind'] = ii
#     row['x_min'], row['y_min'], row['z_min'] = np.array(np.where(aa)).min(1)
#     row['x_max'], row['y_max'], row['z_max'] = np.array(np.where(aa)).max(1)
#     row['x_mean'], row['y_mean'], row['z_mean'] = np.array(np.where(aa)).mean(1)
#     atlas_dfs.append(row)
# atlas_dfs = pd.DataFrame(atlas_dfs)
# atlas_df = atlas_dfs.sort_values('y_min', ascending=False).reset_index(drop=True).reset_index().rename(columns={'index':'new_index'})
# atlas_df['match_ind'] = atlas_df.orig_ind + 1000
# atlas_df['new_index'] +=1
# atlas_df.loc[atlas_df['match_ind'] == 1000, 'new_index']=0
# sorted_dat = atlas_dat + 1000
# for ii, row in atlas_df.iterrows():
#     sorted_dat[sorted_dat == row.match_ind] = row.new_index
# sorted_atlas = image.new_img_like(atlas, sorted_dat, atlas.affine, atlas.header)
# uvs = np.unique(sorted_dat)
# atlas_in_4d = np.zeros(list(atlas.shape) + [len(uvs)-1], dtype=int)
# for ii, uv in enumerate(uvs):
#     if uv != 0:
#         atlas_in_4d[:,:,:,ii-1] = (sorted_dat == uv).astype(int)
# sorted_4d_img = image.new_img_like(atlas,atlas_in_4d, affine=atlas.affine,copy_header=True)


# row = dat_paths.iloc[0]
    
# img = image.load_img(row.image_path)
# regions_extracted_img = resample_to_img(sorted_4d_img,img,interpolation='nearest')

In [55]:
#regions_extracted_img.to_filename('/data/MBDU/midla/notebooks/campcc/basc_atlas_resampled_v2.nii.gz')

In [5]:
regions_extracted_img = image.load_img('/data/MBDU/midla/notebooks/campcc/basc_atlas_resampled_v2.nii.gz')

In [4]:
dat_paths

,subject,session,task,run,total_n_session,scanner,acq_date,confounds_path,image_path,mask_path
0,21748,1,mid,2,4,3TC,2018-11-10,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
1,21748,2,mid,2,4,3TC,2019-03-23,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
2,21748,3,mid,2,4,3TC,2019-07-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
3,22228,1,mid,1,3,3TA,2019-04-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
4,22228,2,mid,1,3,3TA,2019-09-07,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
5,22228,3,mid,1,3,3TA,2020-01-04,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/se

In [5]:
dat_paths

,subject,session,task,run,total_n_session,scanner,acq_date,confounds_path,image_path,mask_path
0,21748,1,mid,2,4,3TC,2018-11-10,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-01/func/sub-21748_ses-01_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
1,21748,2,mid,2,4,3TC,2019-03-23,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-02/func/sub-21748_ses-02_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
2,21748,3,mid,2,4,3TC,2019-07-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-21748/out/fmriprep/sub-21748/ses-03/func/sub-21748_ses-03_task-mid_run-2_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
3,22228,1,mid,1,3,3TA,2019-04-20,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-01/func/sub-22228_ses-01_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
4,22228,2,mid,1,3,3TA,2019-09-07,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-02/func/sub-22228_ses-02_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-brain_mask.nii.gz
5,22228,3,mid,1,3,3TA,2020-01-04,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_desc-confounds_regressors.tsv,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/ses-03/func/sub-22228_ses-03_task-mid_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz,/data/MBDU/midla/data/derivatives/fmriprep/fmriprepv20.1.0_20200528_2mm_clifix_noaroma/sub-22228/out/fmriprep/sub-22228/se

In [39]:
! export PATH=/data/MBDU/midla/env/bin:$PATH; python /data/MBDU/midla/notebooks/campcc/ts_extract.py /data/MBDU/midla/notebooks/campcc/for_chris.csv /data/MBDU/midla/notebooks/campcc/basc_atlas_resampled_v2.nii.gz 0 ifc ccm

/data/MBDU/midla/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
ifc_ccm_0
Extracting task timeseries from row 0
enter function
load image
compute mask
region mask


In [22]:
cfd_dict = {0:'ccm',
           1:'ccmp'}

In [44]:
cmd_list = []
atlas = '/data/MBDU/midla/notebooks/campcc/basc_atlas_resampled_v2.nii.gz'
dat_dir = '/data/MBDU/midla/notebooks/campcc/for_chris.csv'
regions_extracted_img = image.load_img(atlas)
for i in ['bgc','ifc']:
    for j in cfd_dict:
        cfd = cfd_dict[j]
        for _,row in dat_paths.iterrows():
            cmd_list.append(f'! export PATH=/data/MBDU/midla/env/bin:$PATH; python ts_extract.py {dat_dir} {atlas} {_} {i} {cfd}')

In [48]:
cmds = '\n'.join(cmd_list)

from pathlib import Path
swarm_file = Path('/data/MBDU/midla/notebooks/campcc/swarms/ts_extract_v2_test.txt')
swarm_file.write_text(cmds)

run_name = 'ts_extract_v2_test'
swarm_log_dir = '/data/MBDU/midla/notebooks/campcc/swarms/logs'

In [49]:
! swarm -f $swarm_file -g 25 -t 2 -b 15 --time 0:16:00 --logdir $swarm_log_dir --partition=quick --job-name $run_name

59593578


In [11]:
sub_df = pd.DataFrame(dat_paths.subject.unique()).rename(columns={0:'subject'})
boots = pd.DataFrame(np.random.randint(0,sub_df.index.max(), size=[len(sub_df), 10000]), columns=[f'boot_{nn:05d}' for nn in np.arange(1,10001)])
boots = pd.concat([pd.DataFrame(sub_df.index.values, columns=['boot_00000']),boots],axis = 1)
for i in range(boots.shape[0]):
    for j in range(boots.shape[1]):
        boots.loc[i][j] = sub_df.loc[boots.loc[i][j]]

In [14]:
boots.to_csv('/data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstrap.csv',index=None)

In [20]:
! export PATH=/data/MBDU/midla/env/bin:$PATH; python run_bootstrap.py 296 /data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstrapped_matrices ccmp /data/MBDU/midla/notebooks/campcc/for_chris.csv /data/MBDU/midla/notebooks/campcc/mid_v2/subject_ts ifc /data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstraps

/data/MBDU/midla/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
testing
passed, running
not rs file
run_bootstrap.py:185: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (corr_matrices != []):
run_bootstrap.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dat_paths['unique_subject'][_] = row.unique_subject+10000
R[write to console]: boundary (singular) fit: see ?isSingular

R[

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSingular

R[write to console]: boundary (singular) fit: see ?isSi

In [23]:
cmd_list = []
corr_dir = '/data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstrapped_matrices'
dat_dir = '/data/MBDU/midla/notebooks/campcc/for_chris.csv'
output_dir = '/data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstraps'
ts_dir = '/data/MBDU/midla/notebooks/campcc/mid_v2/subject_ts'
for i in ['ifc','bgc']:
    for j in cfd_dict:
        cfd = cfd_dict[j]
        for k in range(0,1001):
            cmd_list.append(f'! export PATH=/data/MBDU/midla/env/bin:$PATH; python /data/MBDU/midla/notebooks/campcc/run_bootstrap.py {k} {corr_dir} {cfd} {dat_dir} {ts_dir} {i} {output_dir}')

In [28]:
cmds = '\n'.join(cmd_list)

from pathlib import Path
swarm_file = Path('/data/MBDU/midla/notebooks/campcc/swarms/icc_mid_v2.txt')
swarm_file.write_text(cmds)

run_name = 'icc_mid_v2'
swarm_log_dir = '/data/MBDU/midla/notebooks/campcc/swarms/logs'

In [29]:
! swarm -f $swarm_file -g 25 -t 2 -b 15 --time 0:16:00 --logdir $swarm_log_dir --partition=quick --job-name $run_name

59622003


In [2]:
def load_data(path):
    import pandas as pd
    try:
        dat = pd.read_csv(path, engine='python')
        name_parts = path.as_posix().split("/")[-1].split('_')
        dat['timeseries'] = name_parts[0]
        dat['boot'] = name_parts[3]
        dat['cfd'] = name_parts[-1]
        dat = dat.loc[dat.type == 'ICC2', ['ICC', 'F', 'lower bound', 'upper bound', 'measure', 'timeseries', 'boot', 'cfd']]
        return dat
    except PermissionError:
        os.remove(path)
bs_res_dir = Path('/data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/bootstraps/')
paths = sorted(bs_res_dir.glob('*.csv'))
bs_reses = Parallel(n_jobs=10, verbose=10)(delayed(load_data)(path) for path in paths)
bs_reses = [load_data(path) for path in paths]
bs_res = pd.concat(bs_reses, ignore_index=True)
bs_res.to_csv('/data/MBDU/midla/notebooks/campcc/mid_v2/test_retest/all_bs_results.csv', index=None)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    7.0s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    7.1s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    7.2s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    7.2s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:    7.3s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1904s.) Setting batch_size=2.
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:    7.3s
[Parallel(n_jobs=10)]: Batch computation too fast (0.1575s.) Setting batch_size=4.
[Parallel(n_jobs=10)]: Done  70 tasks      | elapsed:    7.5s
[Parallel(n_jobs=10)]: Done  96 tasks      | elapsed:    7.6s
[Parallel(n_jobs=10)]: Done 152 tasks      | elapsed:    8.1s
[Parallel(n_jobs=10)]: Done 212 tasks      | elapsed:    8.4s
[Parallel(n_jobs=10)]: Done 280 tasks      | elapsed:    9.0s
[Parallel(n_jobs=10)]: Done 348 tasks      | elapsed:    9.3s
[Parallel(n_j

In [7]:
bs_res.groupby(['timeseries',  'cfd', 'boot',])[['ICC']].mean().reset_index().groupby(['timeseries', 'cfd']).agg(['mean', 'std', 'count'])

ICC                
                         mean       std count
timeseries cfd                               
bgc        ccm.csv   0.303784  0.018864  1001
           ccmp.csv  0.303656  0.018890  1001
ifc        ccm.csv   0.394987  0.041010  1001
           ccmp.csv  0.394845  0.041038  1001

In [9]:
bs_res.groupby(['timeseries',  'cfd', 'boot',])[['ICC']].std().reset_index().groupby(['timeseries', 'cfd']).agg(['mean', 'std', 'count'])

ICC                
                         mean       std count
timeseries cfd                               
bgc        ccm.csv   0.152781  0.004882  1001
           ccmp.csv  0.152872  0.004888  1001
ifc        ccm.csv   0.115745  0.010591  1001
           ccmp.csv  0.115803  0.010580  1001

In [4]:
bs_res[0:10]

,ICC,F,lower bound,upper bound,measure,timeseries,boot,cfd
0,0.466469,6.280649,0.316398,0.630576,1_0,bgc,00000,ccm.csv
1,0.520845,7.542372,0.371530,0.676412,2_0,bgc,00000,ccm.csv
2,0.493244,6.914329,0.343377,0.653343,2_1,bgc,00000,ccm.csv
3,0.456858,6.046841,0.306693,0.622367,3_0,bgc,00000,ccm.csv
4,0.564178,8.767095,0.417370,0.711353,3_1,bgc,00000,ccm.csv
5,0.412858,5.333934,0.264800,0.582835,3_2,bgc,00000,ccm.csv
6,0.425201,5.487565,0.276285,0.594174,4_0,bgc,00000,ccm.csv
7,0.366211,4.485104,0.220933,0.539846,4_1,bgc,00000,ccm.csv
8,0.337546,4.057228,0.194821,0.512430,4_2,bgc,00000,ccm.csv
9,0.377802,4.747998,0.232147,0.550369,4_3,bgc,00000,ccm.csv
